In [20]:
import pandas as pd
import datetime as dt
import numpy as np

from csv import writer

from tensorflow.keras.models import Sequential
from keras.layers import GRU, Dense, Dropout,Bidirectional
from keras.models import load_model

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

df = pd.read_csv('../../../data/clean/btc_open.csv')

df['H-L'] = df['high'] - df['low']
MA_1 = 7
MA_2 = 14
MA_3 = 21

df[f'SMA_{MA_1}_open'] = df['open'].rolling(window=MA_1).mean()
df[f'SMA_{MA_2}_open'] = df['open'].rolling(window=MA_2).mean()
df[f'SMA_{MA_3}_open'] = df['open'].rolling(window=MA_3).mean()

df[f'SD_{MA_1}_open'] = df['open'].rolling(window=MA_1).std()
df[f'SD_{MA_3}_open'] = df['open'].rolling(window=MA_3).std()
df.dropna(inplace=True)

pre_day =1
scala_x = MinMaxScaler(feature_range=(0,1))
scala_y = MinMaxScaler(feature_range=(0,1))
cols_x = ['high', 'low', 'close', 'H-L', f'SMA_{MA_1}_open', f'SMA_{MA_2}_open', f'SMA_{MA_3}_open', f'SD_{MA_1}_open',f'SD_{MA_3}_open']
cols_y = ['open']
scaled_data_x = df[cols_x].values.reshape(-1, len(cols_x))
scaled_data_y = df[cols_y].values.reshape(-1, len(cols_y))

x_total = []
y_total = []


for i in range(pre_day, len(df)):
    x_total.append(scaled_data_x[i-pre_day:i])
    y_total.append(scaled_data_y[i])

    # TEST SIZE
test_size = (int)(len(scaled_data_y) * 0.2)
print(test_size)

x_train = np.array(x_total)
x_test = np.array(x_total[len(x_total)-test_size:])
y_train = np.array(y_total)
y_test = np.array(y_total[len(y_total)-test_size:])


311


In [21]:

model = Sequential()

model.add(GRU(units=128, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(64, activation='tanh'))
model.add(GRU(units=64, return_sequences=True, input_shape=(1, len(cols_x))))
# model.add(Dropout(0.2))
# model.add(Dense(32, activation='sigmoid'))
model.add(Bidirectional(GRU(units=8, return_sequences=False)))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['accuracy'])
model.fit(x_train, y_train, epochs=120, steps_per_epoch=40, use_multiprocessing=True)
# ,validation_data=(x_test, y_test)
model.save('../../../models/GRU/GRU_OPEN.h5')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)


Epoch 1/120
40/40 [==============================] - 8s 4ms/step - loss: 853716992.0000 - accuracy: 0.0000e+00
Epoch 2/120
40/40 [==============================] - 0s 4ms/step - loss: 853593600.0000 - accuracy: 0.0000e+00
Epoch 3/120
40/40 [==============================] - 0s 4ms/step - loss: 853542400.0000 - accuracy: 0.0000e+00
Epoch 4/120
40/40 [==============================] - 0s 4ms/step - loss: 853502208.0000 - accuracy: 0.0000e+00
Epoch 5/120
40/40 [==============================] - 0s 4ms/step - loss: 853465472.0000 - accuracy: 0.0000e+00
Epoch 6/120
40/40 [==============================] - 0s 4ms/step - loss: 853429504.0000 - accuracy: 0.0000e+00
Epoch 7/120
40/40 [==============================] - 0s 4ms/step - loss: 853394944.0000 - accuracy: 0.0000e+00
Epoch 8/120
40/40 [==============================] - 0s 4ms/step - loss: 853360448.0000 - accuracy: 0.0000e+00
Epoch 9/120
40/40 [==============================] - 0s 6ms/step - loss: 853326528.0000 - accuracy: 0.0000e+00
E

In [ ]:
# Testing
predict_price = model.predict(np.array([scaled_data_x[0:]]).reshape(-1, 1, len(cols_x)))
print(predict_price)
